In [ ]:
# !find ./log/* -type f -not -name "stepNum*" -exec rm -f {} \;
# clean the json format log and it's corresponding csv file

In [ ]:
import pandas
import matplotlib.pyplot as plt
import json
import matplotlib
# 设置字体为 Times New Roman
matplotlib.rcParams['font.family'] = 'Times New Roman'
# 配色设置
# COLORS = ['red', 'blue', 'green', 'yellow', 'black', 'purple', 'pink', 'orange', 'gray', 'brown', 'cyan', 'magenta',
#           'olive', 'lime', 'teal', 'navy']
# COLORS = plt.get_cmap('tab20c').COLORS
# cite: https://sashamaps.net/docs/resources/20-COLORS/ 最好分辨的20种颜色
COLORS = ['#e6194B', '#3cb44b', '#f58231', 'blue', '#ffe119', '#f032e6', '#42d4f4', '#fabed4', '#469990', 'purple', '#9A6324', '#000075', '#800000', '#aaffc3', '#fffac8', '#a9a9a9', '#ffffff', '#000000']
print("COLORS_len:", len(COLORS))
plt.figure(figsize=(10, 1))
for i in range(len(COLORS)):
    plt.fill_between([i, i + 1], 0, 1, color=COLORS[i])
plt.show()
# 作图设置
LABLE_SIZE = 30
TICK_SIZE = 25
SHOW_LEGEND = False

# 文件导出位置
SUB_DIR = "longterm/"

In [ ]:
class Log():
    def __init__(self, json_log_path):
        self.json_log_path = json_log_path
        self.param = json.load(open(self.json_log_path, 'r'))
        self.param_names = list(self.param.keys())
    def __getitem__(self, key):
        assert key in self.param.keys(), "key not in param"
        return self.param[key]
    
    def get_data(self,nrows=None):
        if nrows is None:
            return pandas.read_csv(self.param["data"])
        else:
            return pandas.read_csv(self.param["data"], nrows=nrows)
    
    def __str__(self) -> str:
        return f"Log({self.json_log_path})"
    
    def serialize(self) -> str:
        """
        序列化的时候忽略属性"other"
        """
        temp_json = self.param.copy()
        temp_json.pop("other")
        temp_json.pop("data")
        return json.dumps(temp_json)
    

# log1 = Log("./log/20231224063327_11cd96b4-ca81-4e25-ad9e-5aec0c3f3fa3.json")
# print(log1.param_names, log1["normId"])
# log1["data"]
# print(log1.serialize())
# log1.get_data()
# del log1

In [ ]:
import pandas
import os

NROWS = 200000
logs = list()
for root, dirs, files in os.walk("./log/"):
    for file in files:
        if file.endswith(".json"):
            logs.append(Log(os.path.join(root, file))) 
            print(os.path.join(root, file))
print(logs)

# 对logs按照norm的大小排序
logs = sorted(logs, key=lambda log:log["normId"])

# 限制读取行数为 100000 行
data = logs[0].get_data(NROWS)
data

In [ ]:
# sort logs by normId
logs = sorted(logs, key=lambda log:log["normId"])

print("logs_num:", len(logs))
filtered_logs = list()
for log in logs:
    if log["stepNum"] == 50000 and log['mu'] == 0.001 and log['p0'] == 0.5 and log["payoffMatrix"] == "payoffMatrix_longterm_no_norm_error" and log["gamma"] == 1:
    # if log["stepNum"] == 1000000 and log["mu"] == 0.0001 and log["p0"] == 0.5:
    # if log["stepNum"] == 1000000 and log["mu"] == 0.0001 and log["p0"] == 1.0 and log["payoffMatrix"] == "payoffMatrix_longterm_no_norm_error":
    # if log["stepNum"] == 100000 and log["mu"] == 0.0001 and log["p0"] == 1.0 and log["payoffMatrix"] == "payoffMatrix_longterm_no_norm_error":
        # 时间
        time_str = log['data'].split("/")[-1].split("_")[0]
        if int(time_str[:8]) >= 20240321:
            print(log.serialize())
            filtered_logs.append(log)
    LOG_STEP = log['other']['logStep']
print("filtered_logs_num:", len(filtered_logs))

In [ ]:
end = NROWS
# 如果原数据作图是 x in 0~50000 则 zoom_ratio = 4/5 将只画出 x in 0~40000 的数据
data_zoom_ratio = 1
for log in filtered_logs:
    # 读取数据
    data = log.get_data(NROWS)
    # 第一列为横轴，第二列为纵轴画曲线，label为列名
    # 设置cmap

    plt.figure(figsize=(10, 5))
    for i in range(1, 17):
        plt.plot(data.iloc[:end, 0], data.iloc[:end, i], label=data.columns.values[i], color=COLORS[i - 1])
    # 刻度间隔
    # x_major_locator = plt.MultipleLocator(5000)
    # y_major_locator = plt.MultipleLocator(0.1)
    ax = plt.gca()
    # ax.xaxis.set_major_locator(x_major_locator)
    # ax.yaxis.set_major_locator(y_major_locator)
    # 设置刻度字符大小
    ax.tick_params(axis='both', which='major', labelsize=TICK_SIZE)
    # 设置刻度
    margin = 0.02
    plt.xlim(-log["stepNum"]*data_zoom_ratio *margin, log["stepNum"]* data_zoom_ratio*(1+margin))
    plt.ylim(-1*margin*2, 1 + margin*2)
    
    plt.legend().remove()
    plt.xlabel('Time', fontsize=LABLE_SIZE)
    plt.ylabel('Strategies\nabundance', fontsize=LABLE_SIZE)
    # plt.title("Social norm ID: " + str(log["normId"]), fontsize=LABLE_SIZE)
    # plt.title("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
    plt.tight_layout()
    plt.savefig('./temp_fig/' + SUB_DIR + 'd-r_norm' + str(log["normId"]) + '.pdf', dpi=600, bbox_inches='tight', format='pdf')
    plt.savefig('./temp_fig/' + SUB_DIR + 'd-r_norm' + str(log["normId"]) + '.png', dpi=600, bbox_inches='tight', format="png")

# 留存图例
fig = plt.figure()
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='center', fontsize=LABLE_SIZE, ncol = 8)

fig.savefig('./temp_fig/' + SUB_DIR + 'd-r_norm' + '_legend' + '.pdf', dpi=600, bbox_inches='tight', format='pdf')
fig.savefig('./temp_fig/' + SUB_DIR + 'd-r_norm' + '_legend' + '.png', dpi=600, bbox_inches='tight', format="png")
plt.close(fig)   

In [ ]:
end = NROWS
plt.figure(figsize=(10, 5))
idx = 0
for log in filtered_logs:
    # 读取数据
    data = log.get_data(NROWS)
    # 第一列为横轴，第二列为纵轴画曲线，label为列名
    # 设置cmap
    i = 6
    label = "norm" + str(log["normId"])
    plt.plot(data.iloc[:end, 0], data.iloc[:end, i], label=label, color=COLORS[idx])
    idx+=1
    # 刻度间隔
    x_major_locator = plt.MultipleLocator(5000)
    y_major_locator = plt.MultipleLocator(0.1)
    ax = plt.gca()
    ax.xaxis.set_major_locator(x_major_locator)
    ax.yaxis.set_major_locator(y_major_locator)
    # 设置刻度
    margin = 0.02
    plt.xlim(-log["stepNum"]*margin, log["stepNum"]*(1+margin))
    plt.ylim(-1*margin*2, 1 + margin*2)
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.20), ncol=8, fancybox=True, shadow=True)
    plt.xlabel('Time', fontsize=15)
    plt.ylabel('DISC-SR\nabundance', fontsize=15)
    # plt.title("Social Norm Id: " + str(log["normId"]), fontsize=15)
    # plt.title("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
    plt.tight_layout()
    # plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)
plt.savefig('./temp_fig/' + SUB_DIR + 'd-r_norm' + "16_norms" + '.pdf', dpi=600, format='pdf', bbox_inches='tight')
plt.savefig('./temp_fig/' + SUB_DIR + 'd-r_norm' + "16_norms" + '.png', dpi=600, format='png', bbox_inches='tight')

In [ ]:
for log in filtered_logs:
    # 读取数据
    data = log.get_data(NROWS)
    # 第一列为横轴，第二列为纵轴画曲线，label为列名
    fig, axes = plt.subplots(1,2,figsize=(20,5))
    # fig.suptitle("Social norm ID: " + str(log["normId"]), fontsize=LABLE_SIZE)
    # 紧凑布局
    # fig.suptitle("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
    for i in range(17, 21):
        color_index = i - 17
        axes[0].plot(data.iloc[:end, 0], data.iloc[:end, i], label=data.columns.values[i], color=COLORS[color_index])
        axes[0].set_title("Donor", fontsize=LABLE_SIZE)
        # axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=4, fancybox=True, shadow=True)
        # axes[0].legend(loc='center right')
        axes[0].legend(loc='center right', fontsize=TICK_SIZE)
        axes[0].set_xlabel('Time', fontsize=LABLE_SIZE)
        axes[0].set_ylabel('Abundance', fontsize=LABLE_SIZE)
        axes[0].yaxis.set_tick_params(labelsize=TICK_SIZE)
        axes[0].xaxis.set_tick_params(labelsize=TICK_SIZE)
        axes[1].plot(data.iloc[:end, 0], data.iloc[:end, i+4], label=data.columns.values[i+4], color=COLORS[color_index])
        axes[1].set_title("Recipient", fontsize=LABLE_SIZE)
        # axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=4, fancybox=True, shadow=True)
        axes[1].legend(loc='center right', fontsize=TICK_SIZE)
        axes[1].set_xlabel('Time', fontsize=LABLE_SIZE)
        axes[1].yaxis.set_tick_params(labelsize=TICK_SIZE)
        axes[1].xaxis.set_tick_params(labelsize=TICK_SIZE)
        # axes[1].set_ylabel('Abundance', fontsize=LABLE_SIZE)

    fig.tight_layout()
    plt.savefig('./temp_fig/' + SUB_DIR + 'd_r_norm' + str(log['normId']) + '.pdf', dpi=600, bbox_inches='tight', format='pdf')
    plt.savefig('./temp_fig/' + SUB_DIR + 'd_r_norm' + str(log['normId']) + '.png', dpi=600, bbox_inches='tight', format='png')

# 留存图例
fig = plt.figure()
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='center', fontsize=LABLE_SIZE, ncol = 8)

fig.savefig('./temp_fig/' + SUB_DIR + 'd_r_norm' + '_legend' + '.pdf', dpi=600, bbox_inches='tight', format='pdf')
fig.savefig('./temp_fig/' + SUB_DIR + 'd_r_norm' + '_legend' + '.png', dpi=600, bbox_inches='tight', format="png")
plt.close(fig)   

In [ ]:
end = NROWS
plt.figure(figsize=(10, 5))
idx = 0
for log in filtered_logs:
    # 读取数据
    data = log.get_data(NROWS)
    # 第一列为横轴，第二列为纵轴画曲线，label为列名
    i = 22
    this_ylabel = data.columns.values[i] + " abundance"
    label = "norm" + str(log["normId"])
    plt.plot(data.iloc[:end, 0], data.iloc[:end, i], label=label, color=COLORS[idx])
    idx+=1
# 刻度间隔
x_major_locator = plt.MultipleLocator(5000)
y_major_locator = plt.MultipleLocator(0.1)
ax = plt.gca()
ax.xaxis.set_major_locator(x_major_locator)
ax.yaxis.set_major_locator(y_major_locator)
# 设置刻度
margin = 0.02
plt.xlim(-log["stepNum"]*margin, log["stepNum"]*(1+margin))
plt.ylim(-1*margin*2, 1 + margin*2)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.20), ncol=8, fancybox=True, shadow=True)
plt.xlabel('Time', fontsize=15)
# plt.ylabel('DISC-SR\nabundance', fontsize=15)
plt.ylabel(this_ylabel, fontsize=15)
# plt.title("Social Norm Id: " + str(log["normId"]), fontsize=15)
# plt.title("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
plt.tight_layout()
# plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)
plt.savefig('./temp_fig/' + SUB_DIR + 'd-r_norm' + "16_norms" + '.pdf', dpi=600, format='pdf', bbox_inches='tight')
plt.savefig('./temp_fig/' + SUB_DIR + 'd-r_norm' + "16_norms" + '.png', dpi=600, format='png', bbox_inches='tight')

In [ ]:
# 画第25列
for log in filtered_logs:
    # 读取数据
    data = log.get_data(NROWS)
    # 第一列为横轴，第二列为纵轴画曲线，label为列名
    # plt.figure(figsize=(20, 6))
    fig, axes = plt.subplots(1,2,figsize=(20, 5))
    # fig.subtitle("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
    # fig.suptitle("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
    # fig.suptitle("Norm Id: " + str(log["normId"]), fontsize=LABLE_SIZE)
    fig.supxlabel("Time", fontsize=LABLE_SIZE)
    # fig.subplots_adjust(top=0.8)
    color_index = 0
    for i in range(25, 27):
        index = i - 25
        if (data.columns.values[i] == 'good_rep'):
            # axes[index].set_xlabel(data.columns.values[0])
            axes[index].set_ylabel("Good reputation", fontsize=LABLE_SIZE)
        elif (data.columns.values[i] == 'cr'):
            axes[index].set_ylabel("Cooperation rate", fontsize=LABLE_SIZE)
        else: 
            raise "error"
        axes[index].plot(data.iloc[:end, 0], data.iloc[:end, i], label=data.columns.values[i], color=COLORS[color_index])
        axes[index].xaxis.set_tick_params(labelsize=TICK_SIZE)
        axes[index].yaxis.set_tick_params(labelsize=TICK_SIZE)
        color_index += 1
    fig.tight_layout()
    
    plt.savefig('./temp_fig/' + SUB_DIR + 'good_rep_cr_norm' + str(log['normId']) + '.pdf', dpi=600, bbox_inches='tight', format='pdf')
    plt.savefig('./temp_fig/' + SUB_DIR + 'good_rep_cr_norm' + str(log['normId']) + '.png', dpi=600, bbox_inches='tight', format='png')


In [ ]:
# 画第25列
for log in filtered_logs:
    # 读取数据
    data = log.get_data(NROWS)
    color_index = 0
    for i in range(25, 27):
        fig = plt.figure(figsize=(10, 5))
        ax = plt.gca()
        # fig.suptitle("Norm Id: " + str(log["normId"]), fontsize=LABLE_SIZE)
        ax.set_xlabel("Time", fontsize=LABLE_SIZE)
        if (data.columns.values[i] == 'good_rep'):
            ax.set_ylabel("Good reputation", fontsize=LABLE_SIZE)
            save_name = 'good_rep_norm'
        elif (data.columns.values[i] == 'cr'):
            ax.set_ylabel("Cooperation rate", fontsize=LABLE_SIZE)
            save_name = 'cr_norm'
        else: 
            raise "error"
        ax.plot(data.iloc[:end, 0], data.iloc[:end, i], label=data.columns.values[i], color=COLORS[color_index])
        ax.xaxis.set_tick_params(labelsize=TICK_SIZE)
        ax.yaxis.set_tick_params(labelsize=TICK_SIZE)
        color_index += 1
        fig.tight_layout()
        
        plt.savefig('./temp_fig/' + SUB_DIR + save_name + str(log['normId']) + '.pdf', dpi=600, bbox_inches='tight', format='pdf')
        plt.savefig('./temp_fig/' + SUB_DIR + save_name + str(log['normId']) + '.png', dpi=600, bbox_inches='tight', format='png')

In [ ]:
# 画第25列

fig = plt.figure(figsize=(16, 8))
color_index = 0
plt.xlabel("Time")
for log in filtered_logs:
    # 读取数据
    data = log.get_data(NROWS)
    # 第一列为横轴，第二列为纵轴画曲线，label为列名
    # plt.figure(figsize=(20, 6))
    # fig.subtitle("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
    # fig.suptitle("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
    # fig.subplots_adjust(top=0.8)
    
    i = 26
    index = i - 25
    plt.ylabel("Good Reputation")
    plt.plot(data.iloc[:end, 0], data.iloc[:end, i], color=COLORS[color_index], label="norm" + str(log["normId"]))
    color_index += 1
    # plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.30), ncol=4, fancybox=True, shadow=True)
    # plt.title()
    # plt.savefig('./norm10-c-ur.png', dpi=600)

# 刻度间隔
x_major_locator = plt.MultipleLocator(5000)
y_major_locator = plt.MultipleLocator(0.1)
ax = plt.gca()
ax.xaxis.set_major_locator(x_major_locator)
ax.yaxis.set_major_locator(y_major_locator)
margin = 0.02
plt.xlim(-log["stepNum"] * margin, log["stepNum"] * (1 + margin))
plt.ylim(-1 * margin * 2, 1 + margin * 2)
plt.xticks(fontsize=TICK_SIZE)
plt.yticks(fontsize=TICK_SIZE)
plt.xlabel("Time", fontsize=LABLE_SIZE)
plt.ylabel("Cooperation rate", fontsize=LABLE_SIZE)
plt.tight_layout()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.20), ncol=8, fancybox=True, shadow=True, fontsize=TICK_SIZE - 8)
plt.tight_layout()
plt.savefig('./temp_fig/' + SUB_DIR + 'cr_all_norm' + '.pdf', dpi=600, format='pdf', bbox_inches='tight')
plt.savefig('./temp_fig/' + SUB_DIR + 'cr_all_norm' + '.png', dpi=600, format='png', bbox_inches='tight')

In [ ]:
# !jupyter nbconvert --to markdown visualize_json_longterm.ipynb

In [ ]:
# # 用pandoc 转为 pdf
# !pandoc visualize_json_longterm.md -o visualize_json_longterm.pdf --pdf-engine=xelatex  -V geometry:margin=1in 